In [3]:
#conexion twitter y neo4j
from twython import Twython
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
import pandas as pd
import pprint as pp
APP_KEY = 'rXbK97bg6LvBpH9WqDJkqw10T'
APP_SECRET = 'nRGOOhI3sCzsNMLpd5drXmHRAhvT9BOi9HN2vtCI3975CUZUHe'

twitter = Twython(APP_KEY, APP_SECRET)

OAUTH_TOKEN = '976279920279261184-f5RUiD4PRLmOzVQV8FzfXJ827bKwLvI'
OAUTH_TOKEN_SECRET = 'sUk7XhsexQWJ6cmcu3cSs8YgBWYSvFBbjiMrazXTgpNs0'

twitter = Twython(APP_KEY, APP_SECRET,OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
db = GraphDatabase("http://localhost:7474", username="neo4j", password="root")

In [15]:
#cargando los datos de twitter 
result =twitter.search(q='ValdiviaCL', count =100)
datos = []
for i in range(0,len(result["statuses"])):
    usuario_men =[]
    for j in range(0,len(result["statuses"][i]['entities']['user_mentions'])):
        usuario_men.append({'usuario': result['statuses'][i]['entities']['user_mentions'][j]['name']})
    dato = {
        'texto': resultados['statuses'][i]['text'],
        'usuario': resultados['statuses'][i]['user']['name'],
        'usuarios_men': usuario_men
    }
    datos.append(dato)

In [16]:
#crear listas con los usuarios y los twitts
usuarios = []
usuariosGraph = []
usuariosNeo = db.labels.create("Usuarios")
for d in datos:
    for c in d['usuarios_men']:
        usuarios.append(c['usuario'])
    usuarios.append(d['usuario'])
usuarios = list(set(usuarios))

tweets = []
tweetsGraph = []
tweetsNeo = db.labels.create("Tweets")
for d in datos:
    tweets.append(d['texto'])

In [17]:
#crea los nodos para cada usuario y por cada twitt
for u in range(len(usuarios)):
    usuariosGraph.append(db.nodes.create(Usuario=usuarios[u]))
    usuariosNeo.add(usuariosGraph[u])
for t in range(len(tweets)):
    tweetsGraph.append(db.nodes.create(Tweet=tweets[t]))
    tweetsNeo.add(tweetsGraph[t])

In [19]:
#crear las relaciones
for n in datos:
    iTweet = tweets.index(d['texto'])
    iUsuario = usuarios.index(d['usuario'])
    tweetsGraph[iTweet].relationships.create("twitteado por", usuariosGraph[iUsuario])
    for m in d['usuarios_men']:
        iMencionado = usuarios.index(m['usuario'])
        tweetsGraph[iTweet].relationships.create("Menciona a", usuariosGraph[iMencionado])